## Importing libraries

In [ ]:
# import sqlalchemy
from google.cloud import bigquery
from sqlalchemy.engine import create_engine
from google.oauth2 import service_account

import numpy as np
import pandas as pd
import re
import geopy.distance

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
if 5>float('inf'):
    print('d')

## Data reading 

In [ ]:
credentials_json_path = "credentials_view.json"

credentials = service_account.Credentials.from_service_account_file(
    credentials_json_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bqclient = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [ ]:
# PROJECT_ID = credentials.project_id
# DB_NAME = "house_data"
# DATASET = "london_house_prices"

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string(
    "candidate-testing.house_data.london_house_prices"
)
rows = bqclient.list_rows(
    table
)
df = rows.to_dataframe()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# df['address'].value_counts()

In [ ]:
# df[df.address=='66, Nansen Road, London, Greater London SW11 5NT']

## Data cleaning and preprocessing

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df.shape

In [ ]:
df.columns

In [ ]:
df = df[~((df.latitude==0) & ((df.longitude==0)))]
df.shape

In [ ]:
#if everything is same except type we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='type'], keep=False)

#if everything is same except bedrooms we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='bedrooms'], keep=False)

#if everything is same except latitude and longitude we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='latitude' and col!='longitude'])

#if everything is same except area we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='area'], keep=False)

#if everything is same except price we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='price'], keep=False)

#if everything is same except tenure we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='tenure'], keep=False)

#if everything is same except is_newbuild we have to drop that rows
df = df.drop_duplicates([col for col in df.columns if col!='is_newbuild'], keep=False)

In [ ]:
df.shape

In [ ]:
df_row = df.iloc[5]

df_row

In [ ]:
import torch

In [ ]:
df_row

In [ ]:
df_row.drop('price').values

In [ ]:
torch.tensor(float(df_row['price']))

In [ ]:
df.sample(1)

## Feature engineering

### Address column

In [ ]:
# # df[df['address'].str.contains("'")]
# # df.loc[1383]['address']

# s = "Flat 14, 1, King's Arms Court, London, Greater London E1 1AQ"
# s.lower().replace("'","")

In [ ]:
df['address'] = df['address'].apply(lambda x: x.lower().replace("'",""))

In [ ]:
sample_sentence_1 = '6, Polworth, london, Greater London SW16 2EU'.lower()
sample_sentence_2 = '6, Polworth Street, London, Greater London SW16 2EU'.lower()

In [ ]:
def extract_road(x):
    """This function extracts the road/street/... from input string
    """
    road_matcher = re.compile(r'(\w+\s?\w+),\s?london,')
    res = road_matcher.findall(x)
    if res:
        return res[0]
    return None

print(extract_road(sample_sentence_1))
print(extract_road(sample_sentence_2))

In [ ]:
df['road'] = df['address'].apply(extract_road)

In [ ]:
df['road'].value_counts()

In [ ]:
df = df[~df['road'].isna()].reset_index(drop=True)
df.shape

In [ ]:
# uniuqe_ll = df[['latitude','longitude']].drop_duplicates().values
# uniuqe_ll

In [ ]:
# k = 0
# for each in uniuqe_ll:
#     res = df[(df.latitude==each[0]) & (df.longitude==each[1])]
#     if res['road'].nunique()>1:
# #         print(res['road'].unique())
# #         print('------')
#         k+=1
        
# #         if k==40:
# #             break

In [ ]:
df.head(2)

### Date column

In [ ]:
df.date

In [ ]:
max(df.date)

In [ ]:
min(df.date)

In [ ]:
today = pd.to_datetime(np.datetime64('today')).tz_localize(None)

In [ ]:
df['date'] = df['date'].dt.tz_localize(None)

In [ ]:
df['date_now'] = today

In [ ]:
df['days_from_now'] = (df.date_now - df.date).dt.days

In [ ]:
df.sample(4)

### Latitude Longitude to kilometers from center

In [ ]:
center = [51.509865, -0.118092]
example = [51.51092, 0.06250]
geopy.distance.distance(center, example).km

In [ ]:
def distance_from_center(coordinate):
    latitude = float(coordinate.split(',')[0])
    longitude = float(coordinate.split(',')[1])
    center = [51.509865, -0.118092]
    coordinate = [latitude, longitude]
    distance = geopy.distance.distance(center, coordinate).km
    return distance

In [ ]:
df['coordinate'] = df['latitude'].astype(str) + ',' + df['longitude'].astype(str)

In [ ]:
df['coordinate']

In [ ]:
distance_from_center(df.coordinate[0])

In [ ]:
df['distance_from_center'] = df.coordinate.apply(distance_from_center)

In [ ]:
df.distance_from_center.max()

## Train test split and one_hot_encode

In [ ]:
df.columns

In [ ]:
data = df.drop(['address','date','date_now','coordinate'],axis=1)

In [ ]:
X = data.drop(['price'], axis=1)
y = data['price']

In [ ]:
X.shape, y.shape

# Here i have mistake (train one hot then test one hot)

In [ ]:
X.head(3)

In [ ]:
def one_hot_encode(df, column):
    one_hot = pd.get_dummies(df[column])
    df = df.drop(column, axis = 1)
    df = df.join(one_hot)
    return df

In [ ]:
X = one_hot_encode(X, 'type')
X = one_hot_encode(X, 'area')
X = one_hot_encode(X, 'tenure')
X = one_hot_encode(X, 'road')

X.shape

In [ ]:
X.head(2)

In [ ]:
def normalize(df, col):
    return (df[col] - df[col].mean())/df[col].std()

In [ ]:
for col in ['bedrooms', 'latitude', 'longitude', 'days_from_now', 'distance_from_center']:
    X[col] = normalize(X, col)

In [ ]:
X.head(2)

In [ ]:
# batch generator
def get_batches(X, Y, batch_size):
    n_samples = X.shape[0]
        
    # Shuffle at the start of epoch
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
    
        yield X[batch_idx], Y[batch_idx]

In [ ]:
X.to_csv('X.csv', encoding='utf-8', index=False)

In [ ]:
y.to_csv('y.csv', encoding='utf-8', index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
(X_train.shape,y_train.shape), (X_val.shape,y_val.shape), (X_test.shape,y_test.shape)

In [ ]:
X_train.head(3)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42)

In [ ]:
X_train.shape

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
import joblib

In [ ]:
filename = 'finalized_model.sav'
joblib.dump(clf, filename)